### <center> Лекционный материал

In [71]:
import psycopg2
import pandas as pd
import warnings
import requests
from bs4 import BeautifulSoup

warnings.filterwarnings('ignore')

In [2]:
#параметры подключения


In [3]:
connection = psycopg2.connect(
   dbname = DBNAME,
   user = USER,
   host = HOST,
   password = PASSWORD,
   port = PORT
)

In [4]:
n = 10
query = f'''select *
           from sql.pokemon
           limit {n}
        '''

In [5]:
df = pd.read_sql_query(query, connection)
display(df)

DatabaseError: Execution failed on sql 'select *
           from sql.pokemon
           limit 10
        ': relation "sql.pokemon" does not exist
LINE 2:            from sql.pokemon
                        ^


In [ ]:
connection.close()

### <center> Решение задач

In [ ]:
query_3_1 = f'''
        SELECT
            COUNT(id) AS "count"
        
        FROM
            VACANCIES
'''

df = pd.read_sql_query(query_3_1, connection)
display(df)


,count
0,49197


In [ ]:
query_3_2 = f'''
        SELECT
            COUNT(id) AS count
        
        FROM
            EMPLOYERS
'''

df = pd.read_sql_query(query_3_2, connection)
display(df)

,count
0,23501


In [ ]:
query_3_3 = f'''
        SELECT
            COUNT(id) AS count

        FROM
            AREAS
'''

df = pd.read_sql_query(query_3_3, connection)
display(df)

,count
0,1362


In [ ]:
query_3_4 = f'''
        SELECT
            COUNT(id) AS count

        FROM
            INDUSTRIES
'''

df = pd.read_sql_query(query_3_4, connection)
display(df)

,count
0,294


In [ ]:
query_4_1 = f'''
        SELECT
            COUNT(v.id) AS cnt,
            a.name AS area

        FROM
            VACANCIES as v
        LEFT JOIN
            AREAS AS a ON v.area_id = a.id

        GROUP BY
            a.name

        ORDER BY
            COUNT(v.id) DESC

        LIMIT
            5
'''

df = pd.read_sql_query(query_4_1, connection)
display(df)

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


In [ ]:
query_4_2 = f'''
        SELECT
            COUNT(id)
        
        FROM
            VACANCIES
        
        WHERE
            salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

df = pd.read_sql_query(query_4_2, connection)
display(df)

,count
0,24073


In [ ]:
query_4_3 = f'''
        SELECT
            INT4(AVG(salary_from)) AS avg_salary_from,
            INT4(AVG(salary_to)) AS avg_salary_to

        FROM
            VACANCIES        
'''

df = pd.read_sql_query(query_4_3, connection)
df

,avg_salary_from,avg_salary_to
0,71065,110537


In [ ]:
query_4_4 = f'''
        SELECT
            COUNT(id),
            schedule,
            employment

        FROM
            VACANCIES
        
        GROUP BY
            schedule, employment

        ORDER BY
            COUNT(id) DESC

        LIMIT
            3
'''

df = pd.read_sql_query(query_4_4, connection)
display(df)

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость


In [ ]:
query_4_5 = f'''
        SELECT
            COUNT(id),
            experience
        
        FROM
            VACANCIES
        
        GROUP BY
            experience

        ORDER BY
            COUNT(id)
'''

df = pd.read_sql_query(query_4_5, connection)
display(df)

,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


In [ ]:
query_5_1 = f'''
        SELECT
            e.name,
            COUNT(v.id)

        FROM
            VACANCIES AS v
        LEFT JOIN
            EMPLOYERS AS e ON v.employer_id = e.id

        GROUP BY
            e.name
        
        ORDER BY
            COUNT(v.id) DESC

        LIMIT 
            5
'''

df = pd.read_sql_query(query_5_1, connection)
display(df)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


In [ ]:
query_5_2 = f'''
        SELECT 
            a.name,
            COUNT(DISTINCT e.id) AS "Count employers",
            COUNT(DISTINCT v.id) AS "Count vacancies"

        FROM
            AREAS AS a
        LEFT JOIN
            EMPLOYERS AS e ON a.id = e.area
        LEFT JOIN
            VACANCIES AS v ON v.area_id = a.id

        WHERE
            v.id IS NULL
                
        GROUP BY
            a.id

        ORDER BY
            COUNT(e.id) DESC

        LIMIT
            2
'''

df = pd.read_sql_query(query_5_2, connection)
display(df)

,name,Count employers,Count vacancies
0,Россия,410,0
1,Казахстан,207,0


In [ ]:
query_5_3 = f'''
        SELECT
            e.name,
            COUNT(DISTINCT a.id) AS "Count areas"

        FROM
            VACANCIES AS v
        LEFT JOIN
            AREAS AS a ON a.id = v.area_id
        LEFT JOIN
            EMPLOYERS AS e ON e.id = v.employer_id

        GROUP BY
            e.id

        ORDER BY
            COUNT(DISTINCT a.id) DESC

        LIMIT 
            2
'''

df = pd.read_sql_query(query_5_3, connection)
display(df)

,name,Count areas
0,Яндекс,181
1,Ростелеком,152


In [ ]:
query_5_4 = f'''
        SELECT
            COUNT(e.id) AS "Count employer's industries"

        FROM
            EMPLOYERS AS e
        LEFT JOIN
            EMPLOYERS_INDUSTRIES AS ei ON e.id = ei.employer_id
        LEFT JOIN
            INDUSTRIES AS i ON ei.industry_id = i.id

        WHERE
            ei.industry_id IS NULL
'''

df = pd.read_sql_query(query_5_4, connection)
display(df)

,Count employer's industries
0,8419


In [6]:
query_5_5 = f'''
        SELECT
            e.name

        FROM
            EMPLOYERS AS e
        LEFT JOIN
            EMPLOYERS_INDUSTRIES AS ei ON e.id = ei.employer_id
        LEFT JOIN
            INDUSTRIES AS i ON ei.industry_id = i.id

        GROUP BY
            e.id

        HAVING
            COUNT(i.id) = 4

        ORDER BY
            e.name

        OFFSET
            2

        LIMIT 1
'''

df = pd.read_sql_query(query_5_5, connection)
display(df)

,name
0,2ГИС


In [15]:
query_5_6 = f'''
        SELECT 
            COUNT(i.name) AS "count employers"

        FROM
            EMPLOYERS AS e
        LEFT JOIN
            EMPLOYERS_INDUSTRIES AS ei ON e.id = ei.employer_id
        LEFT JOIN
            INDUSTRIES AS i ON ei.industry_id = i.id

        WHERE
            i.name = 'Разработка программного обеспечения'
'''

df = pd.read_sql_query(query_5_6, connection)
display(df)

,count employers
0,3553


In [85]:
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url, headers = {'User-Agent' : 'Mozila/5.0'})
page = BeautifulSoup(response.text, 'html.parser')



<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="ru">
<head>
<meta charset="utf-8"/>
<title>Города-миллионеры России — Википедия</title>
<script>document.documentElement.className="client-js";(function(){var cookie=document.cookie.match(/(?:^|; )ruwikimwclientprefs=([^;]+)/);if(cookie){var featureName=cookie[1];document.documentElement.className=document.documentElement.className.replace(featureName+'-enabled',featureName+'-disabled');}}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","январь","февраль","март","апрель","май","июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],"wgRequestId":"48be9738-0983-4fb0-9103-76c082f91e99","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Города-миллионеры_России","wgTitle":"Города-миллионеры России","wgCurRevisionId":129870936,"wgRevisionId":129870936,"w